# SIMPLE AGENT

This simple agent is similar to the tool agent (both of them are based on their past knowledge).
The difference is that this agent doesn't have a  tool and tries to generate a cypher query to retrieve the data to answer the question.
The query is not executed, this was a first step to test how prompt can be useful too.
If you don't provide, for example, the schema of the database the generated query can be general and should be changed to be run in a real database.

In [2]:
from autogen import ConversableAgent, register_function, config_list_from_json
from tools import * # Imports all the functions
import os

In [5]:
# If you are using a llama3.1 local model 
config_list = config_list_from_json(env_or_file="CONFIG_LIST", filter_dict={"model": "gpt-4o"})
config_list[0]["api_key"] = os.environ.get("GITHUB_TOKEN")

# This simple prompt will give some advice to the model on how to do things
prompt = """You are a data scientist that works with Cypher queries.
All you have to do is translate the given answer as Cypher queries.

You have to respect this rules:
- You must generate the easiest query possible in cypher format.
- You must instert all the information you have and where you found them.
- You must be precise. 
- You must re-generate the query if you think it could generates some errors.

When you think the query can be run you can submit it as the final answer.
When you have written your final answer you must send 'TERMINATE'.

Now i will give you some information about the database schema.
- nodes -
(:Movie), Describe a movie that has a title and a plot. It can also have the number of likes.
(:Person), Describe actors and directors. They have a name, a birthday and they may also have the death date.

- relationships -
(:Movie) <-[:DIRECTED]- (:Person)
(:Person) -[:ACTED-IN]-> (:Movie)
(:Person) -[:KNOWS]-> (:Person)
(:Movie) -[:type]-> (:Genre)
"""


In [6]:
# This agent will start the conversation asking a simple question
question = ConversableAgent(
    "question",
    llm_config=False,
    code_execution_config=False,
)

# This agent will try to answer the question 
coder = ConversableAgent(
    "coder",
    system_message = prompt,
    llm_config = {"config_list": config_list},
    code_execution_config=False
)

# This will ask the driver to answer to one question 
question.initiate_chat(
    coder,
    message = "What is the title of a movie that talks about romance?",
    max_turns = 2
)

question (to coder):

What is the title of a movie that talks about romance?

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
[autogen.oai.client: 09-06 14:06:30] {329} WARNING - Model llama3.1 is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.
coder (to question):

Here is the simplest Cypher query possible:
```markdown
MATCH (m:Movie {title: null})-[:type]->(g:Genre)<-[:type]-(gm:{name: "Romance"})
RETURN m.title AS Title
```
TERMINATE

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
question (to coder):



--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
[autogen.oai.client: 09-06 14:07:50] {329} WARNING - Model llama3.1 is not found. The cost will be 0. In your config_list, add field {"price" : [

ChatResult(chat_id=None, chat_history=[{'content': 'What is the title of a movie that talks about romance?', 'role': 'assistant', 'name': 'question'}, {'content': 'Here is the simplest Cypher query possible:\n```markdown\nMATCH (m:Movie {title: null})-[:type]->(g:Genre)<-[:type]-(gm:{name: "Romance"})\nRETURN m.title AS Title\n```\nTERMINATE', 'role': 'user', 'name': 'coder'}, {'content': '', 'role': 'assistant', 'name': 'question'}, {'content': 'Wait... I made a small mistake. We want to query movies with romance genre, not the genres themselves.\n\nHere is the corrected Cypher query:\n```markdown\nMATCH (m:Movie {title: null})-[:type]->(g:Genre {name: "Romance"})-[:type]-(tg)\nRETURN m.title AS Title\n```\nTERMINATE', 'role': 'user', 'name': 'coder'}], summary='Wait... I made a small mistake. We want to query movies with romance genre, not the genres themselves.\n\nHere is the corrected Cypher query:\n```markdown\nMATCH (m:Movie {title: null})-[:type]->(g:Genre {name: "Romance"})-[:t